In [1]:
!pip install transformers

## 1) 데이터 로드

In [2]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=9286b01f664366a855fc2a7b03ab398f29a374228195c2c52cb32ef0c8155c9a
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [3]:
import pandas as pd
import numpy as np
import os
import urllib.request
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import shape_list, BertTokenizer, TFBertModel
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf

In [4]:
 urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv", filename="ner_train_data.csv")
 urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv", filename="ner_test_data.csv")
 urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt", filename="ner_label.txt")

('ner_label.txt', <http.client.HTTPMessage at 0x7d0ffbc9b210>)

In [5]:
train_ner_df = pd.read_csv("ner_train_data.csv")
test_ner_df = pd.read_csv("ner_test_data.csv")

In [6]:
train_ner_df

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B
...,...,...
80995,다른 새들의 경우보다 윗부리가 움직이게 됩니다 .,O O O ANM-B O O O
80996,"연세대, 두 외국 스포츠의 양대 산맥인 두 학교의 발사믹식초생 유럽챔피언 케이스는 ...",ORG-B NUM-B O O NUM-B O NUM-B NUM-I CVL-B CVL-...
80997,) 주자의 역량으로 진루하는 것이기 때문에 꼬치구이가 할 수 있는 적극적인 병장 조...,O CVL-B O O O O CVL-B O O O O O O O O
80998,연간 회수되는 볼은 3만개에 달한다 .,O O CVL-B NUM-B O O


In [7]:
test_ner_df

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...
...,...,...
8995,제가 뉴사우스웨일즈주로 가서 프레드를 도와줘야겠어요 .,O LOC-B O PER-B O O
8996,17쿼터 들어 윤석금의 실점이 폭렬했고 KT&G는 주도권을 잡아갔다 .,NUM-B O PER-B O O ORG-B O O O
8997,"오병욱 신임총재는 그동안 울산수산업협동조합장, 검은콩차 출신이 도맡다시피했던 프로스...",PER-B CVL-B O CVL-B CVL-B O O O CVL-B O O O CV...
8998,신판 ‘프로페셔널의 원칙’은 프로페셔널(전문직업인)으로서의 직업적 근기를 내면있게 ...,O AFW-B AFW-I CVL-B O O O O O


In [8]:
# 문장과 레이블을 각각 분리
train_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]

train_label = [sent.split() for sent in train_ner_df['Tag'].values]
test_label = [sent.split() for sent in test_ner_df['Tag'].values]


In [9]:
print(train_sentence[2])
print(train_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


In [10]:
#개체명 태깅 정보 종류

labels = [label.strip() for label in open("ner_label.txt", 'r', encoding =  'utf8')]
print("개체명 태깅 정보:", labels)
print(len(labels))

개체명 태깅 정보: ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']
29


In [11]:
tag_to_index= {tag : index for index, tag in enumerate(labels)}
index_to_tag = {index : tag for index, tag in enumerate(labels)}

In [12]:
print(index_to_tag)

{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [13]:
tag_size = len(tag_to_index)
print(tag_size)

29


## 2) 전처리

In [14]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [15]:
sent = train_sentence[1]
label = train_label[1]

print(sent, label)
print("레이블 정수 인코딩:", [tag_to_index[idx] for idx in label])
print("문장의 길이:", len(sent), "레이블의 길이:", len(label))

['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"] ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블 정수 인코딩: [0, 1, 2, 0, 1, 0, 1]
문장의 길이: 7 레이블의 길이: 7


In [16]:
#BERT 토크나이저를 사용하여 단어를 서브워드로 분리

tokens = []
for word in sent:
    subword_tokens = tokenizer.tokenize(word)
    tokens.extend(subword_tokens)


print(tokens)
print(label)
print(len(tokens), len(label))

['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
16 7


* 레이블 길이도 문장의 길이와 일치하도록 추가 전처리
* 분리된 첫 단어에는 원래 레이블 부여, 나머디 서브 단어들에는 -100 부여

In [17]:
tokens, label_ids = [], []
for word, label_token in zip(train_sentence[1], train_label[1]):
    subword_tokens = tokenizer.tokenize(word)
    tokens.extend(subword_tokens)
    label_ids.extend([tag_to_index[label_token]] + [-100] * (len(subword_tokens) - 1))

print(tokens)
print("레이블:", ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in label_ids])
print("레이블의 정수 인코딩:", label_ids)
print(len(tokens), len(label_ids))

['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블: ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩: [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
16 16


In [18]:
#전체 전처리
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, pad_token_label_id_for_segment =0, pad_token_id_for_label = -100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [],[],[]

    for example, label in tqdm(zip(examples, labels), total = len(examples)):
        tokens = []
        labels_ids = []
        for word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # [CLS] 와 [SEP] 고려
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count :
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]
        # SEP 추가
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]
        # CLS 추가
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_label_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention_mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token_type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with label length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype = int)
    attention_masks = np.array(attention_masks, dtype = int)
    token_type_ids = np.array(token_type_ids, dtype = int)
    data_labels = np.asarray(data_labels, dtype = int)

    return(input_ids, attention_masks, token_type_ids), data_labels

In [19]:
X_train, y_train = convert_examples_to_features(train_sentence, train_label, max_seq_len = 128, tokenizer = tokenizer)

100%|██████████| 81000/81000 [00:56<00:00, 1423.47it/s]


In [20]:
X_test, y_test = convert_examples_to_features(test_sentence, test_label, max_seq_len = 128, tokenizer = tokenizer)

100%|██████████| 9000/9000 [00:03<00:00, 2589.59it/s]


In [21]:
print("원문:", train_sentence[0])
print("기존 레이블:", train_label[0])
print('토큰화 후 원문:',[tokenizer.decode([word]) for word in X_train[0][0]])
print('토큰화 후 레이블:',['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])
print('정수 인코딩 결과:', X_train[0][0])
print('정수 인코딩 레이블:', y_train[0])

원문: ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기존 레이블: ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
토큰화 후 원문: ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [22]:
print("어텐션 마스크:", X_train[1][0])

어텐션 마스크: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# 4) 모델링

In [23]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
        #손실함수에서 소프트맥스 함수를 구현
        self.classifier = tf.keras.layers.Dense(num_labels, kernel_initializer= tf.keras.initializers.TruncatedNormal(0.02), name = 'classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

# 5) 손실함수에서 -100 레이블은 제외시키기(예제)

In [24]:
labels = tf.constant([[-100,2,1,-100]])
logits = tf.constant([[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15], [0.4,0.5,0.1]])

In [25]:
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


In [26]:
# -100인 경우 False -100이 아닌 경우 True
# logits에서 -100의 위치를 가지는 경우 예측값 제거

reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[1])), active_loss)
print(reduced_logits)

tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


In [27]:
#labels 또한 -100이 아닌 실제값만 남기기
labels = tf.boolean_mask(tf.reshape(labels, (-1, )), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


# 6) 손실 함수 구현하기

In [28]:
def compute_loss(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits = True, reduction = tf.keras.losses.Reduction.NONE) #소프트맥스 미사용 시 from_logtis = True
    # -100을 가진 정수는 오차 반영하지 않도록 labels 수정
    active_loss = tf.reshape(labels, (-1, )) != -100

    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

    return loss_fn(labels, reduced_logits)

# 7) 학습

In [29]:
model = TFBertForTokenClassification("klue/bert-base", num_labels = tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)
model.compile(optimizer = optimizer, loss = compute_loss)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [30]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
        label_list = []
        pred_list = []

        for i in range(0, len(label_ids)):
            label_tag = []
            pred_tag = []

            for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
                if label_index != -100:
                    label_tag.append(index_to_tag[label_index])
                    pred_tag.append(index_to_tag[pred_index])

            label_list.append(label_tag)
            pred_list.append(pred_tag)
        return label_list, pred_list

    def on_epoch_end(self, epoch, logs = {}):
        y_predicted = self.model.predict(self.X_test)
        y_predicted = np.argmax(y_predicted, axis = 2)

        label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
        score = f1_score(label_list, pred_list, suffix = True)
        print(' - f1: {:04.2f}'.format(score * 100))
        print(classification_report(label_list, pred_list, suffix = True))


In [31]:
f1_score_report = F1score(X_test, y_test)

In [32]:
model.fit(X_train, y_train, epochs = 3, batch_size = 64, callbacks = [f1_score_report])

Epoch 1/3
282/282 ━━━━━━━━━━━━━━━━━━━━ 82s 276ms/step
 - f1: 51.84


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         AFW       0.10      0.00      0.00       394
         ANM       0.75      0.06      0.11       701
         CVL       0.53      0.34      0.42      5758
         DAT       0.61      0.45      0.52      2521
         EVT       0.37      0.19      0.25      1094
         FLD       1.00      0.00      0.01       228
         LOC       0.59      0.38      0.46      2126
         MAT       0.00      0.00      0.00        12
         NUM       0.74      0.64      0.69      5590
         ORG       0.57      0.49      0.52      4086
         PER       0.69      0.65      0.67      4426
         PLT       0.00      0.00      0.00        34
         TIM       0.40      0.01      0.01       314
         TRM       0.57      0.16      0.25      1964

   micro avg       0.63      0.44      0.52     29248
   macro avg       0.49      0.24      0.28     29248
weighted avg       0.61      0.44      0.50     29248

1266/1266 ━━━━━━━━━━━━━━━

# 8) 예측

학습 단계에서는 레이블이 있지만 예측에서는 레이블 없다.

-100의 위치를 기록해주는 label_mask를 만들어줌


In [33]:
def convert_prediction(examples, max_seq_len, tokenizer, pad_token_id_for_segment = 0, pad_token_id_for_label = -100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [],[]

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0] + [pad_token_id_for_label] * (len(subword_tokens) -1 ))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask =[1] * len(input_id)
        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention_mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token_type_id length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with label_mask length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype = int)
    attention_masks = np.array(attention_masks, dtype = int)
    token_type_ids = np.array(token_type_ids, dtype = int)
    label_masks = np.asarray(label_masks, dtype = int)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [34]:
X_pred, label_masks = convert_prediction(test_sentence, max_seq_len = 128, tokenizer = tokenizer)

100%|██████████| 9000/9000 [00:10<00:00, 819.11it/s] 


In [37]:
print("기존원문:", test_sentence[0])
print("-" * 50)
print("토큰화 후 원문:", [tokenizer.decode([word]) for word in X_pred[0][0]])
print("레이블 마스크:", ['[PAD]' if idx == -100  else '[FIRST]' for idx in label_masks[0]])

기존원문: ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토큰화 후 원문: ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [39]:
def ner_prediction(examples, max_seq_len, tokenizer):
    examples = [sent.split() for sent in examples]
    X_pred, label_masks = convert_prediction(examples, max_seq_len=128, tokenizer = tokenizer)
    y_predicted = model.predict(X_pred)
    y_predicted = np.argmax(y_predicted, axis = 2)

    pred_list, result_list =[], []

    for i in range(0, len(label_masks)):
        pred_tag = []
        for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
            if label_index != -100:
                pred_tag.append(index_to_tag[pred_index])

        pred_list.append(pred_tag)

    for example, pred in zip(examples, pred_list):
        one_sample_result = []
        for one_word, label_token in zip(example, pred):
            one_sample_result.append((one_word, label_token))
        result_list.append(one_sample_result)

    return result_list

In [42]:
sent1= '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2= '하이신사에 속한 섬들도 위로 솟아있는데 타인은 살고있어요'
test_samples =[sent1, sent2]
result_list= ner_prediction(test_samples,max_seq_len=128,tokenizer=tokenizer)
result_list

100%|██████████| 2/2 [00:00<00:00, 1524.92it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아있는데', 'O'),
  ('타인은', 'O'),
  ('살고있어요', 'O')]]